In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/drive/My Drive/ML and AI/Datasets/jena_climate/jena_climate_2009_2016.csv")
print("Length of dataset :",len(df))
print()
display(df.head())
values = df.iloc[:,1:] # excluded date
float_data = np.asarray(values) # converting in numpy array

Length of dataset : 420551



,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


In [3]:
# Normalizing data

mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std

In [4]:
# Making genrator

def generator(data, lookback, delay, min_index, max_index,shuffle=False, batch_size=128, step=6):
    if max_index is None: #If max index is not specified, we will set it to very end of the data
        max_index = len(data) - delay - 1 # 1 step peche kharehoke agay ka data pridict kara sakte hai
    i = min_index + lookback # 0 se start nahe kersakte lookback kerna hai to utna agay se stat kerna parega
    while 1:
        if shuffle: 
            #Generate random rows
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index: 
                i = min_index + lookback    # ager data khatam ho raha ho to yeah set kerdo
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows) # 
        samples = np.zeros((len(rows),lookback // step,data.shape[-1])) # past data
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets # sample pe lookback aur target mai delay ka bht sara data hai

In [5]:
lookback = 1440   # 1440/6 = 240(samples) kitna past data dekhna hai
step = 6          # 10 min bt ka data hai, 6 step matlab 60 min(1 hour) ke bt ka data lo
delay = 144
batch_size = 128  # kitne line uthane hai

train_gen = generator(float_data,
                        lookback=lookback,
                        delay=delay,
                        min_index=0,
                        max_index=200000,
                        shuffle=True,
                        step=step,
                        batch_size=batch_size)

val_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=200001,
                    max_index=300000,
                    step=step,
                    batch_size=batch_size)

test_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=300001,
                    max_index=None,
                    step=step,
                    batch_size=batch_size)

val_steps = (300000 - 200001 - lookback) // batch_size # in book code batch size was not divided
# How many steps to draw from val_gen in order to see the entire validation set

test_steps = (len(float_data) - 300001 - lookback) // batch_size # in book code batch size was not divided
# How many steps to draw from test_gen in order to see the entire test set

# Training and evaluating a simple 1D convnet on the Jena data

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu',input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

model.compile(optimizer=RMSprop(), loss='mae')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, None, 32)          2272      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 32)          5152      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 32)          5152      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [8]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=20,
                              validation_data=val_gen,
                              validation_steps=val_steps)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
500/500 [==============================] - 12s 25ms/step - loss: 0.4140 - val_loss: 0.4744
Epoch 2/20
500/500 [==============================] - 12s 24ms/step - loss: 0.3571 - val_loss: 0.4660
Epoch 3/20
500/500 [==============================] - 12s 24ms/step - loss: 0.3351 - val_loss: 0.4500
Epoch 4/20
500/500 [==============================] - 12s 24ms/step - loss: 0.3174 - val_loss: 0.4720
Epoch 5/20
500/500 [==============================] - 12s 24ms/step - loss: 0.3035 - val_loss: 0.4681
Epoch 6/20
500/500 [==============================] - 12s 24ms/step - loss: 0.2942 - val_loss: 0.4740
Epoch 7/20
500/500 [==============================] - 12s 24ms/step - loss: 0.2855 - val_loss: 0.4581
Epoch 8/20
500/500 [==============================] - 12s 24ms/step - loss: 0.2792 - val_loss: 0.4931
Epoch 9/20
500/500 [==============================] - 12s 23ms/step - loss: 0.2736 - val_loss: 0.4714
Epoch 

The validation MAE stays in the 0.40s: you can’t even beat the common-sense baseline using the small convnet

1D Convnet is blazing Fast and can be an alternate of rnn but they don't perform well here.

We can also combine 1d Conv with Rnn to process large sequence that was not possible with just using Rnn.
The conv-RNN will turn the long input sequence into much shorter (downsampled) sequences of higher-level features. This sequence of 
extracted features then becomes the input to
the RNN part of the network.


In [9]:
#####################################

For longer sequences, you can either look at data from longer ago (by increasing the lookback parameter of the data gen-
erator) or look at high-resolution timeseries (by decreasing the step parameter of the
generator).

Here, somewhat arbitrarily, you’ll use a step that’s half as large, resulting
in a timeseries twice as long, where the temperature data is sampled at a rate of
1 point per 30 minutes. The example reuses the generator function defined earlier.

In [10]:
# previously step = 6 (10 min bt ka data hai, 6 step matlab 60 min(1 hour) ke bt ka data lo)
# Now step = 3 (3 step matlab 30 minke bt ka data lo))

step = 3
lookback = 720
delay = 144
batch_size = 128

# Model combining a 1D convolutional base and a GRU layer

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu',input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5))
model.add(layers.Dense(1))
model.summary()

model.compile(optimizer=RMSprop(), loss='mae')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, None, 32)          2272      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, None, 32)          5152      
_________________________________________________________________
gru (GRU)                    (None, 32)                6336      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 13,793
Trainable params: 13,793
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=20,
                              validation_data=val_gen,
                              validation_steps=val_steps)

Epoch 1/20
500/500 [==============================] - 140s 280ms/step - loss: 0.3432 - val_loss: 0.3214
Epoch 2/20
500/500 [==============================] - 139s 278ms/step - loss: 0.3064 - val_loss: 0.3010
Epoch 3/20
500/500 [==============================] - 139s 278ms/step - loss: 0.2880 - val_loss: 0.2969
Epoch 4/20
500/500 [==============================] - 137s 274ms/step - loss: 0.2792 - val_loss: 0.3005
Epoch 5/20
500/500 [==============================] - 135s 271ms/step - loss: 0.2695 - val_loss: 0.2903
Epoch 6/20
500/500 [==============================] - 136s 273ms/step - loss: 0.2626 - val_loss: 0.3006
Epoch 7/20
500/500 [==============================] - 135s 271ms/step - loss: 0.2554 - val_loss: 0.2963
Epoch 8/20
500/500 [==============================] - 136s 272ms/step - loss: 0.2484 - val_loss: 0.3016
Epoch 9/20
500/500 [==============================] - 134s 269ms/step - loss: 0.2418 - val_loss: 0.3136
Epoch 10/20
500/500 [==============================] - 133s 267m

Judging from the validation loss, this setup isn’t as good as the regularized GRU alone,
but it’s significantly faster. It looks at twice as much data, which in this case doesn’t
appear to be hugely helpful but may be important for other datasets.

#Wrapping up
Here’s what you should take away from this section:

 In the same way that 2D convnets perform well for processing visual patterns in
2D space, 1D convnets perform well for processing temporal patterns. They
offer a faster alternative to RNN s on some problems, in particular natural-
language processing tasks.

 Typically, 1D convnets are structured much like their 2D equivalents from the
world of computer vision: they consist of stacks of Conv1D layers and Max-
Pooling1D layers, ending in a global pooling operation or flattening operation.

 Because RNN s are extremely expensive for processing very long sequences, but
1D convnets are cheap, it can be a good idea to use a 1D convnet as a prepro-
cessing step before an RNN , shortening the sequence and extracting useful rep-
resentations for the RNN to process